### Import Modules:

In [1]:
import time
import matplotlib.pyplot as plt
import numpy as np
% matplotlib inline
np.random.seed(2018) 

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

C:\Users\Home\Anaconda3\envs\image-classification\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Parameters:

In [3]:
# image dimensions:
img_width, img_height, num_channels = 150, 150, 3

# data directory:
train_data_dir = '../data/train'
validation_data_dir = '../data/validation'

# no. of samples:
nb_train_samples = 2000
nb_validation_samples = 800

# hyperparams
epochs = 100
batch_size = 16

# input shape 
channels_first = K.image_data_format() == 'channels_first'
input_shape = (num_channels, img_width, img_height) if channels_first else (img_width, img_height, num_channels)

### Custom Model:

In [4]:
def Custom_Model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

    return model

### Model Accuracy and Loss:

In [5]:
def plot_stats(model_stats):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_stats.history['acc'])+1),model_stats.history['acc'])
    axs[0].plot(range(1,len(model_stats.history['val_acc'])+1),model_stats.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_stats.history['acc'])+1),len(model_stats.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    
    # summarize history for loss
    axs[1].plot(range(1,len(model_stats.history['loss'])+1),model_stats.history['loss'])
    axs[1].plot(range(1,len(model_stats.history['val_loss'])+1),model_stats.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_stats.history['loss'])+1),len(model_stats.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    
    plt.show()

### Data Augmentation:

In [6]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

# this is a generator that will read pictures found in
# subfolers of '../data/train', and indefinitely generate
# batches of augmented image data.
train_generator = train_datagen.flow_from_directory(
    train_data_dir,  # this is the target directory
    target_size=(img_width, img_height), # all images will be resized to 150x150
    batch_size=batch_size,
    class_mode='binary') # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 802 images belonging to 2 classes.


### Train Model:

In [7]:
model = Custom_Model(input_shape)

### Train the model
'''
# training start time
start = time.time()

# fits the model on data generated batch-by-batch by a python generator.
# the generator is run in parallel to the model, for efficiency. 
# for instance, this allows you to do real-time data augmentation on images on CPU in parallel to training your model on GPU.
model_stats = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

# training end time
end = time.time()

print "Model took %0.2f seconds to train"%(end - start)
'''

'\n# training start time\nstart = time.time()\n\n# fits the model on data generated batch-by-batch by a python generator.\n# the generator is run in parallel to the model, for efficiency. \n# for instance, this allows you to do real-time data augmentation on images on CPU in parallel to training your model on GPU.\nmodel_stats = model.fit_generator(\n    train_generator,\n    steps_per_epoch=nb_train_samples // batch_size,\n    epochs=epochs,\n    validation_data=validation_generator,\n    validation_steps=nb_validation_samples // batch_size)\n\n# training end time\nend = time.time()\n\nprint "Model took %0.2f seconds to train"%(end - start)\n'

### Save Model:

In [8]:
# always save the model after or during training
# model.save_weights('first_try.h5')

### Test Model:

In [9]:
def accuracy(model, validation_generator):
    result = model.predict_generator(validation_generator, verbose=1)
    predicted_class = np.argmax(result, axis=1)
    true_class = np.argmax(test_y, axis=1)
    num_correct = np.sum(predicted_class == true_class) 
    accuracy = float(num_correct)/result.shape[0]
    return (accuracy * 100)

In [10]:
# testing the model on validation set
# print "Accuracy on validation data is: %0.2f"%accuracy(model, validation_generator)